In [1]:
import gradio as gr
from transformers import pipeline
import numpy as np
import openai
import warnings
warnings.filterwarnings('ignore')
import os
from dotenv import load_dotenv
load_dotenv()

openai.api_key = os.getenv("OPENAI_API_KEY")


C:\Users\c_mar\anaconda3\envs\dev\lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [2]:
# Preference
# Initialize the ASR pipeline
transcriber = pipeline("automatic-speech-recognition", model="openai/whisper-base.en")

def transcribe_and_save(audio):
    sr, y = audio
    y = y.astype(np.float32)
    y /= np.max(np.abs(y))
    transcription = transcriber({"sampling_rate": sr, "raw": y})["text"]
    
    # Write the transcription to a file
    with open("transcription_movie_selection.txt", "w") as file:
        file.write(transcription)
    
    return transcription

# Create the Gradio interface
demo = gr.Interface(
    transcribe_and_save,
    gr.Audio(sources=["microphone"]),
    "text",
)

demo.launch(share=False)
#change to True for outside URL


Running on local URL:  http://127.0.0.1:7868

To create a public link, set `share=True` in `launch()`.


In [4]:
#pure movie selection analysis with OpenAI
#preprocess txt file

with open('transcription_movie_selection.txt', 'r', encoding="utf-8") as file:
    text = file.read()


In [5]:
# Load environment variables from .env file
from dotenv import load_dotenv
load_dotenv()

# Set your OpenAI API key
openai.api_key = os.getenv("OPENAI_API_KEY")

# Function time

def get_preference(question):
    messages = [
        {"role": "system", "content": "Determine three movies to make the person happier based on the provided text and provide links to the movie posters from TMDB."},
        {"role": "user", "content": question}
    ]
    
    # Use the chat completions method
    response = openai.chat.completions.create(
        model="gpt-4",
        messages=messages
    )
    
    # Extract the answer from the response
    answer = response.choices[0].message.content
    return answer

def analyze_document(filename):
    with open(filename, 'r') as file:
        text = file.read()
    preference = get_preference(text)
    print(f"Movie Recommendations: {preference}")

analyze_document('transcription_movie_selection.txt')


Movie Recommendations: 1. "The Secret Life of Pets 2" - Even though this movie doesn't contain aliens and marshmallows, it involves adorable dogs and exciting car actions that could bring a smile to your face. Here is the link to the poster: https://www.themoviedb.org/movie/412117-the-secret-life-of-pets-2/posters

2. "Lilo & Stitch" - Aliens and Dogs together make this film fun. There are no cars and marshmallows, but it is a feel-good movie that will surely light up your day.
Here is the link to the poster:
https://www.themoviedb.org/movie/11544-lilo-stitch/posters

3. "Cars" - This movie is all about fast and fun Cars. It doesn't have dogs, aliens, or marshmallows but is definitely an exciting watch. Check the movie poster here:
https://www.themoviedb.org/movie/920-cars/posters

Please note that it's hard to find a movie that contains all these elements (Dogs, Aliens, Marshmallows, and Cars) together. However, these picks could give you a joyful cinematic experience.
